In [1]:
from copy import deepcopy
from collections import defaultdict
import random

## Reading and initializing functions

In [2]:
def get_states(filename, sudokusize=9):
    '''
    get_states is to read multiple sudokus in the format:
    ..8..4..5...64..5 etc.
    '''
    sudokus = []
    with open(filename) as sudoku_states:
        for state in sudoku_states:
            sudoku = []
            row = 1
            for i, content in enumerate(state):
                if i%sudokusize == 0 and i > 0:
                    row+=1
                try:
                    int(content)
                    clause = int(str(row) + str(i%sudokusize+1) + content)
                    sudoku.append(clause)
                except:
                    # is not convertible to int
                    continue
            sudokus.append(sudoku)
    return sudokus

def get_state_from_dimacs(filename, sudokusize=9):
    '''
    This function reads a sudoku state in dimacs format
    '''
    sudoku = []
    with open(filename) as state:
        for clause in state:
            sudoku.append(int(clause.split()[0]))
    return sudoku

def get_rules(filename):
    constraints = {} # dictionary with constraints as: clausenumber->[literals]
    pointers = {} # dictionary with pointers to clause numbers for literals as: literals->[clausenumbers]
    with open(filename) as sudoku_rules:
        for index, clause in enumerate(sudoku_rules):
            if index == 0:
                continue # skip first line
            constraints[index]=[]
            for literal in clause.split():
                lit = int(literal)
                if lit!=0:
                    constraints[index].append(lit)
                    if lit not in pointers.keys():
                        pointers[lit]=[index]
                    else:
                        pointers[lit].append(index)
                    
    return constraints, pointers

## DP helper functions for constraints updates

In [3]:
def assign_literal(constraints, pointers, literal, assignments):
    '''
    assign the given literal to be True.
    assignLiteral is aimed to be one recursion stack, so deepcopies are made
    in order to properly backtrack
    '''
    constraintsCopied = deepcopy(constraints)
    pointersCopied = deepcopy(pointers)
    assignmentsCopied = deepcopy(assignments)
    constraintsUpdated, pointersUpdated = update_constraints(constraintsCopied, pointersCopied, literal)
    assignmentsUpdated = update_assigments(assignmentsCopied, literal)
    return constraintsUpdated, pointersUpdated, assignmentsUpdated
     

def update_constraints(constraints, pointers, literal):
    
    ## Find clauses with literal and remove them
    clauses = pointers[literal]
    for clause in clauses:
        try:
            del constraints[clause]
            pointers[literal].remove(clause)
        except:
#             print("This exception should not occur, but it did")
            continue
            
    ## Find clauses with counter-literal and remove its counter-literal
    clauses = pointers[-literal]
    for clause in clauses:
        try:
            constraints[clause].remove(-literal)
            pointers[-literal].remove(clause)
        except:
#             print("This exception should not occur either, but it did")
            continue
    
    return constraints, pointers

def update_assigments(assignments, literal):
    if literal in assignments:
        print("already assigned", literal, "(in this stack), returning...")
        return None
    else:
        assignments.append(literal)
        return assignments
    
def check_tautologies(constraints, pointers):
    foundOne = False
    for clause in constraints.keys():
        literals = constraints[clause]
        counterLiterals = [-x for x in literals]
        for i in literals:
            for j in counterLiterals:
                if i == j:
                    del constraints[clause]
                    pointers[i].remove(clause)
                    foundOne = True
                    break
                  
    return constraints, pointers, foundOne  

## DP cases

In [4]:
def empty_clause(constraints):
    for clause in constraints.keys():
        if not constraints[clause]:
            return True
        
    return False


def unit_clause(constraints): 
    for clause in constraints.keys():
        if len(constraints[clause])==1:
            literal = constraints[clause][0]
            return literal

    return False

def pure_literal(literals):
    for l in literals:
        if -l not in literals:
            return l
        
    return False

def random_literal(literals): 
    literal = random.choice(literals)
      
    return literal

## DP main function

In [5]:
def DP(constraints, pointers, assignments):
    
    if len(constraints)==0:
        return True
   
    if empty_clause(constraints):
        return False

    literals = list(pointers.keys())
   
    literal = unit_clause(constraints)
    if literal:
        constraints, pointers, assignments = assign_literal(constraints, pointers, literal, assignments)
        DP(constraints, pointers, assignments)
        
    literal = pure_literal(literals)
    if literal:
        constraints, pointers, assignments = assign_literal(constraints, pointers, literal, assignments)
        DP(constraints, pointers, assignments)
                
    literal = random_literal(literals)
    constraints, pointers, assignments = assign_literal(constraints, pointers, literal, assignments)
    if DP(constraints, pointers, assignments):
        return True
    else:
        constraints, pointers, assignments = assign_literal(constraints, pointers, -literal, assignments)
        DP(constraints, pointers, assignments)

In [ ]:

def set_up():
#     state = get_state_from_dimacs('sudoku-example.txt')
    state = get_states('1000 sudokus.txt')[2]
    print("Sudoku initial board state", state)
    constraints, pointers = get_rules('sudoku-rules.txt')
    print("Length of clauses:", len(constraints))
    ## Update contraints according to initial board state
    initial_assignments = []
    for literal in state:
        constraints, pointers, initial_assignments = assign_literal(constraints, pointers, literal, initial_assignments)
    print("clauses after board state:", len(constraints.keys()))
    constraints, pointers, foundOne = check_tautologies(constraints, pointers)
    if foundOne:
        print("Found tautologies and removed corresponding clauses")
    else:
        print("No tautologies found")
    print("Starting DP algorithm...")
    succes = DP(constraints, pointers, initial_assignments)
    if succes:
        print("Solution found")
    else:
        print("No solution found")
        
set_up()
    

Sudoku initial board state [167, 229, 261, 354, 365, 396, 452, 523, 536, 574, 581, 615, 678, 699, 794, 851, 868, 928, 931, 945, 983, 992]
Length of clauses: 11988
clauses after board state: 11944
No tautologies found
Starting DP algorithm...
